# 1. Load packages

In [1]:
# modules included with Anaconda installation
import sys
import numpy as np
import pandas as pd
import pickle
import itertools
import string
import matplotlib.pyplot as plt
from os.path import join
from glob import glob
from pathlib import Path
import ipympl
%matplotlib ipympl

# custom modules
from calc_cum import condense
from fca_lambda import fit_cumulants_lambda, get_cumulants_lambda

# 2. Load spectra

In [2]:
refspec_folder = r'D:\Data\FFS\210417 mG mY mCh2 1p controls'
refspec_fname  = 'emission spectra w masks.csv'
reference_spectra = pd.read_csv(join(refspec_folder, refspec_fname))

# Define fitting parameters

In [3]:
B = np.array([reference_spectra['mEGFP'],
              reference_spectra['mEYFP'],
              reference_spectra['mCherry2']])

# each row corresponds to ones species
# first term in each row is the minimum
# second term is the initial guess
# third term is the maximum
N0 = np.array([[0, 10, 1000],
              [0,10,1000],
              [0,10,1000],
              [0,10,1000],
               [0,10,1000],
               [0,10,1000],
               [0,10,1000]])

# each row is the brightness of each component 
eps0 = np.array([[0.123622893, 0, 0],
                [0, 0.067974408, 0],
                [0, 0, 0.060135561],
                [0.05, 0.05,0],
                [0.05,0,0.05,],
                [0, 0.05, 0.05],
                [0.05,0.05,0.05]])

epsvary = np.array([[1, 0, 0],
                   [0, 1, 0],
                   [0, 0, 1],
                   [0,0,0],
                   [0,0,0],
                   [0,0,0],
                    [0,0,0]])

constr = [[None, None, None],
          [None, None, None],
          [None, None, None],
          ['eps1A', 'eps2B', None],
          ['eps1A', None, 'eps3C'],
         [None, 'eps2B', 'eps3C'],
         ['eps1A', 'eps2B', 'eps3C']]

gamma = np.array([1, 1, 1, 1])
pa = None
pv = None
numcomp = len(N0)
numfl = B.shape[0]

fitpars = {'B': B,
          'N0': N0,
          'eps0': eps0,
          'epsvary': epsvary,
          'constr': constr,
          'gamma': gamma,
          'pa': pa,
          'pv': pv,
          'numcomp': numcomp,
          'numfl': numfl}

# Perform fitting

In [4]:
savetag = 'cumulant fitting output - 7 components variable brightness'

Nlabels = ['N'+str(icomp) for icomp in range(1,numcomp+1)]
epslabels = ['eps'+str(pair[0])+pair[1] for pair in itertools.product(range(1,numcomp+1), string.ascii_uppercase[:numfl])]
collabels = ['M'] + Nlabels + epslabels + ['chi2']

In [23]:
folder = r'D:\Data\FFS\210717 epinephrine stim\epinephrine\unmixed images\masks\cumulants - with filtering'
pattern = '?? * unmixed ROI?? kappa1.npy'
kappa1_files = glob(join(folder, pattern))
labels = [Path(file).stem.split(' kappa1')[0] for file in kappa1_files]

output = pd.DataFrame(index=labels, columns=collabels)

for file, label in zip(kappa1_files, labels):
    # load cumulants
    print(label)
    kappa1 = np.load(file)
    kappa2 = np.load(join(folder, label+' kappa2.npy'))
    var1   = np.load(join(folder, label+' var1.npy'))
    var2   = np.load(join(folder, label+' var2.npy'))
    
    # calculate fit
    fit = fit_cumulants_lambda(N0,eps0,
                           B,
                           kappa1, var1,
                           kappa2, var2,
                           gamma,
                           epsvary=epsvary,
                           Pa=pa,
                           constr=constr,
                           Pavary=pv)
    
    # populate columns in output data frame
    for Nlab in Nlabels:
        output[Nlab][label] = fit.params[Nlab].value
    for epslab in epslabels:
        output[epslab][label] = fit.params[epslab].value
    output['chi2'][label] = fit.redchi
    
# save output csv and fitting parameters
output.to_csv(join(folder, savetag+'.csv'))
fp = open(join(folder, savetag+' parameters.pkl'), 'wb')
pickle.dump(fitpars, fp)
fp.close()

output

01 post unmixed ROI01
01 pre unmixed ROI01
02 post unmixed ROI01
02 pre unmixed ROI01
03 post unmixed ROI01
03 pre unmixed ROI01
04 post unmixed ROI01
04 pre unmixed ROI01
05 post unmixed ROI01
05 pre unmixed ROI01
06 post unmixed ROI01
06 pre unmixed ROI01
07 post unmixed ROI01
07 pre unmixed ROI01
08 post unmixed ROI01
08 pre unmixed ROI01
09 post unmixed ROI01
09 pre unmixed ROI01
10 post unmixed ROI01
10 pre unmixed ROI01
11 post unmixed ROI01
11 pre unmixed ROI01
12 post unmixed ROI01
12 pre unmixed ROI01
13 post unmixed ROI01
13 pre unmixed ROI01
14 post unmixed ROI01
14 pre unmixed ROI01
15 post unmixed ROI01
15 pre unmixed ROI01


,M,N1,N2,N3,N4,N5,N6,N7,eps1A,eps1B,...,eps5A,eps5B,eps5C,eps6A,eps6B,eps6C,eps7A,eps7B,eps7C,chi2
01 post unmixed ROI01,NaN,43.16147,8.466568,54.578532,10.838338,2.298618,7.127677,0.982202,0.081313,0.0,...,0.081313,0.0,0.094135,0.0,0.035914,0.094135,0.081313,0.035914,0.094135,789.097722
01 pre unmixed ROI01,NaN,43.653672,4.65703,53.103277,7.289258,2.508954,5.847108,0.567786,0.093569,0.0,...,0.093569,0.0,0.109302,0.0,0.053111,0.109302,0.093569,0.053111,0.109302,4119.555339
02 post unmixed ROI01,NaN,13.303485,4.821017,31.575613,0.874066,0.54556,1.185352,0.000216,0.098276,0.0,...,0.098276,0.0,0.085041,0.0,0.059906,0.085041,0.098276,0.059906,0.085041,2665.017737
02 pre unmixed ROI01,NaN,15.255933,3.576042,34.377288,0.72453,0.0,1.154268,0.000006,0.101831,0.0,...,0.101831,0.0,0.09013,0.0,0.074701,0.09013,0.101831,0.074701,0.09013,1458.396158
03 post unmixed ROI01,NaN,3.868911,4.065795,38.250549,0.90761,0.0,1.742152,0.000003,0.08895,0.0,...,0.08895,0.0,0.094263,0.0,0.075856,0.094263,0.08895,0.075856,0.094263,323.582884
03 pre unmixed ROI01,NaN,5.671078,4.296457,42.555824,0.001132,0.000573,0.757354,0.000007,0.072985,0.0,...,0.072985,0.0,0.09645,0.0,0.101328,0.09645,0.072985,0.101328,0.09645,1367.929882
04 post unmixed ROI01,NaN,12.558814,11.365785,51.812125,0.068297,0.000001,2.794308,0.000004,0.106516,0.0,...,0.106516,0.0,0.111327,0.0,0.085644,0.111327,0.106516,0.085644,0.111327,330.26874
04 pre unmixed ROI01,NaN,12.28633,9.546275,53.951684,0.006243,0.01654,3.759138,0.000069,0.103215,0.0,...,0.103215,0.0,0.112148,0.0,0.090741,0.112148,0.103215,0.090741,0.112148,344.81623
05 post unmixed ROI01,NaN,24.024928,18.33071,89.349987,0.089515,0.019252,3.675544,0.002345,0.090259,0.0,...,0.090259,0.0,0.095189,0.0,0.06939,0.095189,0.090259,0.06939,0.095189,1276.512676
05 pre unmixed ROI01,NaN,17.858635,12.147244,91.970948,6.883043,1.439818,10.586388,0.377975,0.088297,0.0,...,0.088297,0.0,0.108247,0.0,0.059021,0.108247,0.088297,0.059021,0.108247,11945.215778


In [79]:
output.astype('float').describe()

,M,N1,N2,N3,N4,N5,N6,N7,eps1A,eps1B,...,eps5A,eps5B,eps5C,eps6A,eps6B,eps6C,eps7A,eps7B,eps7C,chi2
count,0.0,1.900000e+01,19.000000,19.000000,1.900000e+01,1.900000e+01,19.000000,1.900000e+01,19.000000,19.0,...,19.000000,19.0,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,NaN,1.607213e+01,21.927023,19.738794,5.910585e-01,1.642467e-01,25.181089,6.586664e-02,0.118429,0.0,...,0.118429,0.0,0.076436,0.0,0.083706,0.076436,0.118429,0.083706,0.076436,2567.476275
std,NaN,1.035360e+01,19.277569,19.374097,9.801745e-01,4.036573e-01,22.296800,1.923008e-01,0.020521,0.0,...,0.020521,0.0,0.051453,0.0,0.016931,0.051453,0.020521,0.016931,0.051453,6247.262637
min,NaN,2.647882e-10,2.736468,2.371410,1.665335e-13,9.936496e-12,0.651531,1.221245e-12,0.068096,0.0,...,0.068096,0.0,0.035862,0.0,0.046236,0.035862,0.068096,0.046236,0.035862,9.976106
25%,NaN,8.789855e+00,10.459729,9.275076,7.900433e-03,3.485736e-06,10.239088,3.442924e-06,0.104170,0.0,...,0.104170,0.0,0.055550,0.0,0.073348,0.055550,0.104170,0.073348,0.055550,339.785679
50%,NaN,1.580857e+01,16.836833,16.057801,5.467219e-02,1.377743e-03,20.016470,2.450970e-04,0.122544,0.0,...,0.122544,0.0,0.062935,0.0,0.080975,0.062935,0.122544,0.080975,0.062935,487.456961
75%,NaN,2.377990e+01,25.680541,22.579486,5.244241e-01,5.443558e-02,32.015470,1.520591e-02,0.134178,0.0,...,0.134178,0.0,0.078777,0.0,0.095887,0.078777,0.134178,0.095887,0.078777,785.773925
max,NaN,3.536197e+01,83.547286,87.869056,3.009664e+00,1.688456e+00,90.670761,7.943272e-01,0.148000,0.0,...,0.148000,0.0,0.281443,0.0,0.116932,0.281443,0.148000,0.116932,0.281443,24080.978942


In [10]:
output.to_csv(join(folder, savetag+'.csv'))

In [87]:
output[['eps1A', 'eps2B', 'eps3C']].astype('float').describe()

,eps1A,eps2B,eps3C
count,20.000000,2.000000e+01,20.000000
mean,0.111928,6.133901e-02,0.059608
std,0.029109,3.889671e-02,0.014644
min,0.043336,3.330669e-14,0.028102
25%,0.099353,3.466737e-02,0.048955
50%,0.119731,6.594750e-02,0.064061
75%,0.133572,8.729671e-02,0.070806
max,0.144979,1.467460e-01,0.079743


In [88]:
output[['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7']].astype('float').describe()

,N1,N2,N3,N4,N5,N6,N7
count,20.000000,2.000000e+01,20.000000,2.000000e+01,20.000000,2.000000e+01,2.000000e+01
mean,24.318361,2.106291e+01,16.952837,1.031641e+00,22.140928,2.558285e+00,1.918986e+00
std,31.574563,4.492831e+01,19.666164,2.259910e+00,25.643234,4.628383e+00,4.035644e+00
min,3.746422,6.259437e-10,0.898391,5.551115e-14,2.313189,5.551115e-14,5.551115e-14
25%,5.508047,3.298445e-04,5.136505,8.457034e-06,7.227619,3.887725e-03,4.747412e-03
50%,14.669796,1.008118e+00,10.603222,2.884226e-02,14.407473,5.772040e-01,2.035004e-01
75%,30.241546,1.222898e+01,19.438628,1.301047e+00,26.138443,1.829247e+00,1.059126e+00
max,145.868813,1.748321e+02,86.612112,9.695603e+00,119.559490,1.488828e+01,1.598823e+01


In [82]:
output[['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7']]

,N1,N2,N3,N4,N5,N6,N7
01 unmixed ROI01,0.0,25.282784,27.99957,0.0,0.0,45.230197,0.0
02 unmixed ROI01,2.70611,34.057848,24.555938,0.003273,0.000679,47.445079,0.000003
03 unmixed ROI01,21.601287,8.198389,10.141448,0.342637,0.001378,9.16125,0.000039
04 unmixed ROI01,12.155505,20.560384,21.204766,0.04622,0.0,23.538755,0.000009
05 unmixed ROI01,22.994887,12.382947,9.846313,0.634572,0.000002,14.528179,0.000004
06 unmixed ROI01,35.361966,26.078299,23.954205,2.307729,0.566948,30.324314,0.045884
07 unmixed ROI01,27.457227,16.836833,18.548396,0.011484,0.000053,22.517051,0.000381
08 unmixed ROI01,28.512927,16.459878,17.973045,3.009664,0.211736,33.706625,0.794327
09 unmixed ROI01,28.786689,18.899407,16.057801,0.054672,0.468716,23.287457,0.337125
10 unmixed ROI01,24.564907,36.057164,4.217704,2.588254,1.688456,0.651531,0.017134


In [83]:
print(
    (output['N1'] / (output['N1']+output['N4']+output['N5']+output['N7'])).values[:20].mean(), '\n',
    (output['N4'] / (output['N1']+output['N4']+output['N5']+output['N7'])).values[:20].mean(), '\n',
    (output['N5'] / (output['N1']+output['N4']+output['N5']+output['N7'])).values[:20].mean(), '\n',
    (output['N7'] / (output['N1']+output['N4']+output['N5']+output['N7'])).values[:20].mean(),
)    

0.9184747264238569 
 0.02746705994169978 
 0.0497126348650193 
 0.0043455787694241625


In [84]:
print(
    (output['N2'] / (output['N2']+output['N4']+output['N6']+output['N7'])).values[:20].mean(), '\n',
    (output['N4'] / (output['N2']+output['N4']+output['N6']+output['N7'])).values[:20].mean(), '\n',
    (output['N6'] / (output['N2']+output['N4']+output['N6']+output['N7'])).values[:20].mean(), '\n',
    (output['N7'] / (output['N2']+output['N4']+output['N6']+output['N7'])).values[:20].mean(),
)    

0.4647445181796529 
 0.016704880396695918 
 0.5171591483911563 
 0.001391453032494743


In [85]:
print(
    (output['N3'] / (output['N3']+output['N5']+output['N6']+output['N7'])).values[:20].mean(), '\n',
    (output['N5'] / (output['N3']+output['N5']+output['N6']+output['N7'])).values[:20].mean(), '\n',
    (output['N6'] / (output['N3']+output['N5']+output['N6']+output['N7'])).values[:20].mean(), '\n',
    (output['N7'] / (output['N3']+output['N5']+output['N6']+output['N7'])).values[:20].mean(),
)    

0.4537503467499935 
 0.015456018192280424 
 0.5292236396702714 
 0.0015699953874547022
